<a href="https://colab.research.google.com/github/microprediction/monteprediction_colab_examples/blob/main/monteprediction_entry_rlang.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
install.packages(c("quantmod", "dplyr", "xts", "randtoolbox", "httr", "data.table"))


Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘zoo’, ‘TTR’, ‘rngWELL’




In [4]:
library(quantmod)
library(dplyr)
library(xts)
library(randtoolbox)
library(httr)
library(data.table)

# Define SPDR ETFs - Example ETFs
MONTE_URL <- 'https://micromonte.pythonanywhere.com/upload'
SPDR_ETFS <- c('XLB', 'XLC', 'XLE', 'XLF', 'XLI', 'XLK', 'XLP', 'XLRE', 'XLU', 'XLV', 'XLY')
KERNEL_RATE <- 300.0

# Utility functions
get_last_wednesday <- function() {
  today <- Sys.Date()
  weekday_today <- match(format(today, "%A"), c("Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"))
  offset <- (weekday_today - 3) %% 7
  last_wednesday <- today - as.difftime(offset, units = "days")
  return(last_wednesday)
}


# Example of fetching financial data and preparing it
last_wednesday <- get_last_wednesday()
num_weeks <- sample(52:(52+4*52), 1)
start_date <- as.Date(last_wednesday) - (7 * num_weeks)

getSymbols(SPDR_ETFS, from = start_date, to = last_wednesday+1, auto.assign = TRUE, periodicity = "weekly")
weekly_prices <- do.call(merge, lapply(SPDR_ETFS, function(sym) Ad(get(sym))))
weekly_returns <- na.omit(ROC(weekly_prices, type = "discrete"))
cov_matrix <- cov(as.matrix(weekly_returns))


# Factory defaults don't change these
num_samples_per_chunk <- 1048576 / 8
num_chunks <- 8
num_samples <- num_chunks * num_samples_per_chunk

# Quasi monte carlo assuming multivariate normal
# Swap this out for your genius method
samples <- sobol(n = num_samples, dim = ncol(weekly_returns))
samples <- samples %*% chol(cov_matrix)
colnames(samples) <- SPDR_ETFS
df <- as.data.frame(samples)

head(df,5)



[1] "XLB"  "XLC"  "XLE"  "XLF"  "XLI"  "XLK"  "XLP"  "XLRE" "XLU"  "XLV" 
[11] "XLY"

,XLB,XLC,XLE,XLF,XLI,XLK,XLP,XLRE,XLU,XLV,XLY
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,0.014548154,0.02207498,0.025818328,0.02322322,0.02141552,0.03150791,0.01886850,0.02902143,0.03050580,0.02528747,0.03520686
2,0.021822231,0.02125601,0.042698611,0.02496451,0.02789168,0.02899544,0.02492875,0.03017906,0.03196641,0.02956896,0.03151914
3,0.007274077,0.02289395,0.008938045,0.02148193,0.01493935,0.03402037,0.01280824,0.02786380,0.02904518,0.02100597,0.03889457
4,0.010911116,0.01655624,0.028393834,0.01405375,0.02008125,0.03381675,0.01105211,0.02637984,0.02038008,0.02316900,0.03035513
5,0.025459270,0.03863122,0.018091811,0.03477099,0.02995956,0.04524975,0.02683262,0.03487357,0.03395953,0.03117999,0.05544402


We need this utility function to submit the samples

In [5]:



# Updated send_in_chunks function to correctly handle the chunks
send_in_chunks <- function(df, email, num_chunks, name = NULL, max_retries = 3) {
  if (is.null(name)) {
    name <- substr(email, 1, 20)
  }

  # Ensure the dataframe is split into the specified number of chunks
  chunks <- split(df, rep(1:num_chunks, each = num_samples_per_chunk, length.out = nrow(df)))

  for (chunk_no in seq_along(chunks)) {
    chunk_df <- chunks[[chunk_no]]
    for (attempt in 1:max_retries) {
      csv_content <- tempfile()
      write.csv(chunk_df, csv_content, row.names = FALSE, quote = TRUE)

      metadata <- list(email = email, name = name, chunk = chunk_no - 1, num_chunks = num_chunks, kernel_rate = KERNEL_RATE)
      response <- POST(MONTE_URL, body = list(file = upload_file(csv_content)), query = metadata, encode = "multipart")

      if (response$status_code == 200) {
        cat(sprintf("Chunk %d of %d sent successfully.\n", chunk_no, num_chunks))
        break
      } else {
        cat(sprintf("Failed to send chunk %d, attempt %d. Response: %s\n", chunk_no, attempt, content(response, "text", encoding = "UTF-8")))
      }
      Sys.sleep(1)  # Delay between retries
    }

    if (attempt == max_retries) {
      cat(sprintf("Failed to send chunk %d after %d attempts.\n", chunk_no, max_retries))
    }
  }
}


In [6]:
# Are you happy?
YOUR_EMAIL <- "rlang_example@microprediction.com"       # Replace with your actual email
YOUR_NAME <- "Your Name"                                # (Optional) Replace with a preferred leaderboard handle
send_in_chunks(df, YOUR_EMAIL, num_chunks, YOUR_NAME)

Chunk 1 of 8 sent successfully.
Chunk 2 of 8 sent successfully.
Chunk 3 of 8 sent successfully.
Chunk 4 of 8 sent successfully.
Chunk 5 of 8 sent successfully.
Chunk 6 of 8 sent successfully.
Chunk 7 of 8 sent successfully.
Chunk 8 of 8 sent successfully.


If this works I suggest converting this to a script and scheduling it to run every weekend.